In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from getData import *

In [ ]:
frames, indices = extract_frames_from_video('(3.) IMG_0177.MOV', (np.logspace(0, 1, 100)-1)/9)
frames = list(map(lambda image: resize_image(image, .2), frames))

In [2]:
prompt = """'Extrae este número en formato float de python, devuelve únicamente el 
número con el formato: "float" y sin ningún otro texto.'"""

In [3]:
def get_gemini_response(frames, prompt):
    """
    Sends a list of frames to Gemini and retrieves the float responses. 
    Args:
        frames (list): List of image frames.
        prompt (str): The prompt to send to Gemini.
    Returns:
        np.ndarray: Array of float responses from Gemini.
    """
    temp = np.zeros(len(frames), dtype=np.float32)
    for ii, frame in enumerate(frames):
        while True:
            try:
                temp[ii] = float(send_image_to_gemini(frame, prompt))
                break
            except Exception as e:
                print(f"Error readying image {ii}: {e}")
                continue
    return temp

In [ ]:
get_gemini_response(frames, prompt):

In [ ]:
plt.plot(np.array(indices[13:-1])/30, temp[13:], 'o-')
plt.xlabel('Tiempo (s)')
plt.ylabel('Temperatura (°C)')

In [ ]:
def func(t, T_0, T_inf, tau):
    return T_inf + (T_0 - T_inf) * np.exp(- t / tau)

from scipy.optimize import curve_fit

In [ ]:
args, covs = curve_fit(func, np.array(indices[13:-1])/30, np.array(temp[13:]), p0=[90, 30, 1])

In [ ]:
plt.plot(np.array(indices[13:-1])/30, temp[13:], 'o')
plt.plot(np.array(indices[13:-1])/30, func(np.array(indices[13:-1])/30, *args), 'r-')
plt.xlabel('Tiempo (s)')
plt.ylabel('Temperatura (°C)')

In [ ]:
args

In [32]:
dataFrame = {}
for ii in range(1, 6):
    dataFrame[f'time_{ii}'] = []
    dataFrame[f'temperature_{ii}'] = []

dataFrame

{'time_1': [],
 'temperature_1': [],
 'time_2': [],
 'temperature_2': [],
 'time_3': [],
 'temperature_3': [],
 'time_4': [],
 'temperature_4': [],
 'time_5': [],
 'temperature_5': []}

In [36]:
data = pd.DataFrame(dataFrame)

# Extract frames from multiple videos and get temperature data
for ii in range(1,6):
    frames, indices = extract_frames_from_video(f'videos/{ii}.MOV', (np.logspace(0, 1, 100)-1)/9)
    frames = list(map(lambda image: resize_image(image, 1), frames))

    temp = get_gemini_response(frames, prompt)
    data[f'time_{ii}'] = np.array(indices[:-1])/30
    data[f'temperature_{ii}'] = temp

    print(f"Video {ii} processed.")
    

Enviando imagen a Gemini para análisis...
Video 5 processed.


In [37]:
data.head()

,time_1,temperature_1,time_2,temperature_2,time_3,temperature_3,time_4,temperature_4,time_5,temperature_5
0,0.000000,80.000000,0.000000,72.599998,0.000000,79.099998,0.000000,73.900002,0.000000,30.700001
1,2.333333,79.800003,2.366667,70.599998,0.933333,78.699997,1.666667,73.400002,4.233333,30.799999
2,4.766667,79.500000,4.833333,70.599998,1.933333,78.300003,3.366667,72.900002,8.566667,30.900000
3,7.233333,79.199997,7.333333,71.000000,2.933333,77.900002,5.100000,72.400002,13.000000,31.000000
4,9.766667,78.900002,9.900000,69.800003,3.933333,77.500000,6.900000,71.800003,17.566667,31.100000


In [43]:
data.to_csv('enfriamiento_data.csv', index=False)